In [132]:
# --- Celda 1: Importación de Librerías ---
import requests
import pandas as pd
import time
import os
import json
import logging
from dotenv import load_dotenv
from collections import deque
import googlemaps # <--- LIBRERÍA NUEVA

logging.info("Librerías importadas.")



2025-10-08 19:56:58,977 - INFO - Librerías importadas.


In [133]:
# --- Celda 2: Configuraciones Básicas ---
# Asegura que no se dupliquen los handlers si se corre la celda múltiples veces
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("001_cali_api.log"),
        logging.StreamHandler()
    ]
)

logging.info("Inicio del notebook 001_cali_api.ipynb.")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
logging.info("Configuraciones de Pandas para visualización aplicadas.")

2025-10-08 19:56:58,988 - INFO - Inicio del notebook 001_cali_api.ipynb.
2025-10-08 19:56:58,989 - INFO - Configuraciones de Pandas para visualización aplicadas.


In [134]:
# --- Celda 3: Carga de Variables de Entorno y Rutas (Modificada) ---

ENV_FILE_PATH = '../.env/.env' 
RAW_API_CSV_PATH = '../datos_brutos/api/001_Local_cali_api.csv'
STATE_FILE_PATH = '../datos_brutos/api/001_Local_cali_api.json'

# --- carga de las claves API ---
logging.info(f"Cargando variables de entorno desde: {ENV_FILE_PATH}")
if os.path.exists(ENV_FILE_PATH):
    load_dotenv(ENV_FILE_PATH)
    logging.info(".env encontrado y cargado.")
else:
    logging.error(f"Archivo .env NO encontrado en {ENV_FILE_PATH}.")
    raise FileNotFoundError(f"Asegúrate de que el archivo .env esté en la ruta correcta: {os.path.abspath(ENV_FILE_PATH)}")

GOOGLE_MAPS_API_KEYS = [os.getenv(key) 
                        for key in os.environ 
                        if key.startswith('GOOGLE_MAPS_API_KEY_')]
GOOGLE_MAPS_API_KEYS = [key for key in GOOGLE_MAPS_API_KEYS if key]
if not GOOGLE_MAPS_API_KEYS:
    raise ValueError("No se encontraron claves API de Google Maps en el .env.")
logging.info(f"{len(GOOGLE_MAPS_API_KEYS)} clave(s) API de Google Maps cargada(s).")

2025-10-08 19:56:59,001 - INFO - Cargando variables de entorno desde: ../.env/.env
2025-10-08 19:56:59,004 - INFO - .env encontrado y cargado.
2025-10-08 19:56:59,005 - INFO - 3 clave(s) API de Google Maps cargada(s).


In [135]:
# --- Celda 4: Gestión de Clientes API y Rotación (SIMPLIFICADA) ---
logging.info("Configurando clientes de la API de Google Maps y lógica de rotación.")

if not GOOGLE_MAPS_API_KEYS:
    raise ValueError("La lista de claves API está vacía. No se pueden crear clientes.")

# Usamos un diccionario para asociar cada cliente con un identificador de su clave
gmaps_clients_map = {
    f"Key_{i+1} (ends ...{key[-4:]})": googlemaps.Client(key=key) 
    for i, key in enumerate(GOOGLE_MAPS_API_KEYS)
}

# La cola ahora contendrá los identificadores de las claves
client_keys_queue = deque(gmaps_clients_map.keys())

logging.info(f"Se crearon {len(gmaps_clients_map)} clientes de Google Maps: {list(client_keys_queue)}")

def get_current_client_id():
    """Devuelve el identificador del cliente activo actual."""
    return client_keys_queue[0]

def get_current_gmaps_client():
    """Devuelve el cliente activo actual."""
    active_key_id = client_keys_queue[0]
    return gmaps_clients_map[active_key_id]

def rotate_gmaps_client():
    """Rota el cliente al final de la cola."""
    if len(client_keys_queue) > 1:
        current_key_id = get_current_client_id()
        logging.warning(f"Rotando cliente de API. La clave '{current_key_id}' puede haber alcanzado su límite.")

        client_keys_queue.rotate(-1) # Mueve el primer elemento al final

        new_key_id = get_current_client_id()
        logging.info(f"Nuevo cliente de API activo: '{new_key_id}'.")
        time.sleep(2) 
    else:
        logging.warning("Intento de rotar, pero solo hay una clave API disponible. Esperando 10 segundos.")
        time.sleep(10) 
        
    return get_current_gmaps_client()

logging.info(f"Gestor de clientes API listo. Cliente inicial activo: '{get_current_client_id()}'.")

2025-10-08 19:56:59,011 - INFO - Configurando clientes de la API de Google Maps y lógica de rotación.
2025-10-08 19:56:59,014 - INFO - API queries_quota: 60
2025-10-08 19:56:59,014 - INFO - API queries_quota: 60
2025-10-08 19:56:59,015 - INFO - API queries_quota: 60
2025-10-08 19:56:59,019 - INFO - Se crearon 3 clientes de Google Maps: ['Key_1 (ends ...oIAc)', 'Key_2 (ends ...dMag)', 'Key_3 (ends ...U_TQ)']
2025-10-08 19:56:59,020 - INFO - Gestor de clientes API listo. Cliente inicial activo: 'Key_1 (ends ...oIAc)'.


In [136]:
# --- Celda 5: Funciones de Extracción (REFACTORIZADA CON WRAPPER CENTRALIZADO) ---
logging.info("Definiendo funciones para buscar y obtener detalles de lugares.")

# --- CONFIGURACIONES CENTRALIZADAS ---
MAX_RETRIES_PER_CALL = 3 # Número de reintentos para errores transitorios
PLACE_DETAILS_FIELDS = [
    'place_id', 'name', 'formatted_address', 'geometry', 'type', 'rating',
    'user_ratings_total', 'reviews', 'price_level', 'opening_hours', 'website',
    'international_phone_number', 'wheelchair_accessible_entrance', 'reservable',
    'business_status'
]

# --- NUEVA FUNCIÓN WRAPPER ---
def make_api_request(api_call_function):
    """
    Función envoltorio que maneja reintentos con backoff exponencial y rotación de claves
    para cualquier llamada a la API de googlemaps.
    
    Args:
        api_call_function (function): Una función lambda que toma un cliente como argumento
                                     y ejecuta la llamada a la API. 
                                     Ej: lambda c: c.places(query="...")
    
    Returns:
        La respuesta de la API si tiene éxito, o None si todos los reintentos y claves fallan.
    """
    initial_client_id = get_current_client_id()
    
    while True: # Bucle principal para la rotación de claves
        client = get_current_gmaps_client()
        client_id = get_current_client_id()
        
        for attempt in range(MAX_RETRIES_PER_CALL): # Bucle para los reintentos
            try:
                # Ejecuta la función de API que se pasó como argumento
                response = api_call_function(client)
                return response # Si tiene éxito, devuelve la respuesta y termina
            
            except googlemaps.exceptions.ApiError as e:
                logging.warning(
                    f"Intento {attempt + 1}/{MAX_RETRIES_PER_CALL} fallido con clave '{client_id}'. "
                    f"Error de API: {e.status}"
                )
                
                # Si el error es de cuota, salimos del bucle de reintentos para rotar la clave inmediatamente
                if e.status == 'OVER_QUERY_LIMIT':
                    break 
                
                # Para otros errores (ej. UNKNOWN_ERROR, TIMEOUT), esperamos y reintentamos
                if attempt < MAX_RETRIES_PER_CALL - 1:
                    wait_time = 2 ** attempt  # Backoff exponencial: 1, 2, 4 seg...
                    logging.info(f"Esperando {wait_time} segundos antes de reintentar...")
                    time.sleep(wait_time)
                else:
                    logging.error(f"Todos los {MAX_RETRIES_PER_CALL} reintentos fallaron para la clave '{client_id}'.")
            
            except Exception as e:
                logging.error(f"Error inesperado durante la llamada API con clave '{client_id}': {e}")
                # Si es un error inesperado, probablemente no sea recuperable, así que no reintentamos
                return None

        # Esta parte solo se alcanza si el bucle de reintentos se rompe (por OVER_QUERY_LIMIT)
        new_client_id = rotate_gmaps_client()
        
        # Verificación para evitar bucles infinitos si todas las claves están agotadas
        if new_client_id == initial_client_id:
            logging.error("Todas las claves API han sido probadas y han fallado. Abortando esta petición.")
            return None

# --- FUNCIONES DE EXTRACCIÓN SIMPLIFICADAS ---

def text_search_paginated(query):
    """
    Realiza una búsqueda de texto paginada usando el wrapper `make_api_request`.
    """
    all_place_ids = []
    
    logging.info(f"Realizando búsqueda inicial para: '{query}'")
    response = make_api_request(lambda c: c.places(query=query))
    
    if not response:
        logging.error(f"La búsqueda inicial para '{query}' falló después de todos los reintentos y rotaciones.")
        return []
        
    all_place_ids.extend([place['place_id'] for place in response.get('results', [])])
    next_page_token = response.get('next_page_token')
    
    while next_page_token:
        logging.info("Paginación: Obteniendo siguiente página de resultados...")
        time.sleep(2) # Pausa obligatoria de Google para la paginación
        
        response = make_api_request(lambda c: c.places(page_token=next_page_token))
        
        if not response:
            logging.error("La paginación falló. Devolviendo los IDs encontrados hasta ahora.")
            break
            
        all_place_ids.extend([place['place_id'] for place in response.get('results', [])])
        next_page_token = response.get('next_page_token')
    
    logging.info(f"Búsqueda para '{query}' completada. Se encontraron {len(all_place_ids)} lugares.")
    return all_place_ids

def get_place_details(place_id):
    """
    Obtiene los detalles de un lugar usando el wrapper `make_api_request`.
    """
    response = make_api_request(
        lambda c: c.place(
            place_id=place_id,
            fields=PLACE_DETAILS_FIELDS,
            language='es'
        )
    )
    # Si la respuesta es None (fallo total) o un dict vacío, devolvemos un dict vacío
    return response.get('result', {}) if response else {}

'''def process_query_and_get_ids(query):
    """
    Orquesta la búsqueda de una consulta completa, manejando la rotación de claves.
    """
    initial_client = get_current_gmaps_client()
    
    while True:
        client = get_current_gmaps_client()
        place_ids = text_search_paginated(query, client)
        
        if place_ids == "API_ERROR":
            rotate_gmaps_client()
            if get_current_gmaps_client() == initial_client:
                logging.error(f"Todas las claves API han fallado para la búsqueda '{query}'.")
                return []
            continue
        
        return place_ids
'''

2025-10-08 19:56:59,036 - INFO - Definiendo funciones para buscar y obtener detalles de lugares.


'def process_query_and_get_ids(query):\n    """\n    Orquesta la búsqueda de una consulta completa, manejando la rotación de claves.\n    """\n    initial_client = get_current_gmaps_client()\n    \n    while True:\n        client = get_current_gmaps_client()\n        place_ids = text_search_paginated(query, client)\n        \n        if place_ids == "API_ERROR":\n            rotate_gmaps_client()\n            if get_current_gmaps_client() == initial_client:\n                logging.error(f"Todas las claves API han fallado para la búsqueda \'{query}\'.")\n                return []\n            continue\n        \n        return place_ids\n'

In [139]:
# --- Celda 5.1: Generador de Consultas Geoespaciales Dinámicas ---

logging.info("Definiendo el sistema escalable de consultas geoespaciales.")

GEOGRAPHICAL_SCOPE = {
    "Colombia": {
        "Valle del Cauca": {
            "cities": ["Buenaventura", "Guadalajara de Buga", "Cartago", "Palmira", "Santiago de Cali", "Tuluá"],
            "towns": ["Alcalá", "Andalucía", "El Ansermanuevo", "Argelia", "Bolívar",  "Bugalagrande", "Caicedonia", "Calima Darién", "Candelaria", "Dagua", "El Águila", "El Cairo", "El Cerrito", "El Dovio", "Florida", "Ginebra", "Guacarí", "Jamundí", "La Cumbre", "La Unión", "La Victoria", "Obando", "Pradera", "Restrepo", "Riofrío", "Roldanillo", "San Pedro", "Sevilla", "Toro", "Trujillo", "Versalles", "Vijes", "Yotoco", "Yumbo", "Zarzal"
            ]
        }
    }
}

SEARCH_QUERIES_TEMPLATES = [
    # --- Gastronomía y Vida Nocturna ---
    "restaurantes gourmet en {location}, {department_state}, {country}",
    "restaurantes de comida típica en {location}, {department_state}, {country}",
    "restaurantes económicos en {location}, {department_state}, {country}",
    "fritanga en {location}, {department_state}, {country}",
    "comida de mar en {location}, {department_state}, {country}",
    "plazas de mercado en {location}, {department_state}, {country}",
    "cafés de especialidad en {location}, {department_state}, {country}",
    "panaderías tradicionales en {location}, {department_state}, {country}",
    "heladerías en {location}, {department_state}, {country}",
    "bares de salsa en {location}, {department_state}, {country}",
    "discotecas y clubes nocturnos en {location}, {department_state}, {country}",
    "escuelas de salsa para turistas en {location}, {department_state}, {country}",
    "bares con música en vivo en {location}, {department_state}, {country}",
    "restaurantes de cocina internacional en {location}, {department_state}, {country}",
    "bares de cocteles y rooftops en {location}, {department_state}, {country}",
    "distritos gastronómicos en {location}, {department_state}, {country}",
    "asaderos de pollo y sancocho en {location}, {department_state}, {country}",
    "fruterías y juguerías en {location}, {department_state}, {country}",
    "tiendas de abarrotes tradicionales en {location}, {department_state}, {country}",
    "cantinas y billares locales en {location}, {department_state}, {country}",
   
    # --- Cultura y Puntos de Interés ---
    "museos en {location}, {department_state}, {country}",
    "teatros y auditorios en {location}, {department_state}, {country}",
    "centros de convenciones en {location}, {department_state}, {country}",
    "estadios y coliseos en {location}, {department_state}, {country}",
    "galerías de arte en {location}, {department_state}, {country}",
    "bibliotecas públicas en {location}, {department_state}, {country}",
    "monumentos históricos en {location}, {department_state}, {country}",
    "parque principal de {location}, {department_state}, {country}",
    "iglesias y templos históricos en {location}, {department_state}, {country}",
    "casa de la cultura en {location}, {department_state}, {country}",
    "centros culturales en {location}, {department_state}, {country}",
    "sitios turísticos en {location}, {department_state}, {country}",

    # --- Alojamiento ---
    "hoteles 5 estrellas en {location}, {department_state}, {country}",
    "hoteles boutique en {location}, {department_state}, {country}",
    "hoteles de cadena internacional en {location}, {department_state}, {country}",
    "apartahoteles y suites en {location}, {department_state}, {country}",
    "hoteles económicos en {location}, {department_state}, {country}",
    "hostales y posadas en {location}, {department_state}, {country}",
    "fincas de alquiler cerca de {location}, {department_state}, {country}",

    # --- Compras y Servicios ---
    "centros comerciales en {location}, {department_state}, {country}",
    "supermercados y grandes superficies en {location}, {department_state}, {country}",
    "casas de cambio en {location}, {department_state}, {country}",
    "consulados y embajadas en {location}, {department_state}, {country}",
    "oficinas gubernamentales en {location}, {department_state}, {country}",
    "centros de salud y hospitales en {location}, {department_state}, {country}",
    "droguerías y farmacias en {location}, {department_state}, {country}",
    "ferreterías en {location}, {department_state}, {country}",
    "colegios y escuelas en {location}, {department_state}, {country}",
    "tiendas de artesanías y souvenirs en {location}, {department_state}, {country}",

    # --- Recreación y Bienestar ---
    "gimnasios y centros deportivos en {location}, {department_state}, {country}",
    "spas y centros de bienestar en {location}, {department_state}, {country}",
    "parques y áreas verdes en {location}, {department_state}, {country}",
    "clubes sociales y deportivos en {location}, {department_state}, {country}",
    "ríos y balnearios cerca de {location}, {department_state}, {country}",
    "canchas deportivas en {location}, {department_state}, {country}",
    "miradores naturales en {location}, {department_state}, {country}",
    "senderos para caminatas en {location}, {department_state}, {country}",
    "parques ecológicos en {location}, {department_state}, {country}",
    "zoológico y jardín botánico en {location}, {department_state}, {country}",
    "piscinas públicas en {location}, {department_state}, {country}",
    "centros de yoga y meditación en {location}, {department_state}, {country}",

    # --- Transporte ---
    "aeropuertos y terminales aéreos en {location}, {department_state}, {country}",
    "terminales de transporte en {location}, {department_state}, {country}",
    "estaciones de transporte masivo en {location}, {department_state}, {country}",
    "agencias de alquiler de vehículos en {location}, {department_state}, {country}",
    "paraderos de transporte rural en {location}, {department_state}, {country}"
]

# ==============================================================================
#  2. Función Generadora de la Lista de Búsqueda Final
# ==============================================================================

MANIFEST_FILE_PATH = '../datos_brutos/api/manifest.json'

def create_and_save_manifest():
    """
    Construye la lista completa de objetos de consulta con el departamento/estado
    incluido en el string de la consulta para máxima precisión.
    """
    manifest_list = []
    
    for country, departments in GEOGRAPHICAL_SCOPE.items():
        for department, locations in departments.items():
            
            # Procesa las ciudades del departamento
            for city in locations.get("cities", []):
                location_with_prefix = f"la ciudad llamada {city}"
                for template in SEARCH_QUERIES_TEMPLATES:
                    # --- CAMBIO CLAVE AQUÍ ---
                    query = template.format(
                        location=location_with_prefix,
                        department_state=department, # Se añade el departamento
                        country=country
                    )
                    query_object = {
                        "query_string": query,
                        "pais": country,
                        "departamento_estado": department,
                        "ciudad": city,
                        "pueblo": None
                    }
                    manifest_list.append(query_object)
            
            # Procesa los pueblos del departamento
            for town in locations.get("towns", []):
                location_with_prefix = f"el pueblo llamado {town}"
                for template in SEARCH_QUERIES_TEMPLATES:
                    # --- CAMBIO CLAVE AQUÍ ---
                    query = template.format(
                        location=location_with_prefix,
                        department_state=department, # Se añade el departamento
                        country=country
                    )
                    query_object = {
                        "query_string": query,
                        "pais": country,
                        "departamento_estado": department,
                        "ciudad": None,
                        "pueblo": town
                    }
                    manifest_list.append(query_object)

    try:
        with open(MANIFEST_FILE_PATH, 'w', encoding='utf-8') as f:
            json.dump(manifest_list, f, indent=4, ensure_ascii=False)
        logging.info(f"Manifiesto de trabajo con {len(manifest_list)} consultas guardado exitosamente en: {MANIFEST_FILE_PATH}")
    except Exception as e:
        logging.error(f"No se pudo guardar el archivo manifest.json: {e}")
        raise

    return manifest_list

# --- Ejecutar la función para generar y guardar el manifiesto final ---
MANIFEST_OF_JOBS = create_and_save_manifest()

# Log de verificación para ver el resultado
if MANIFEST_OF_JOBS:
    logging.info("Ejemplo de consulta final generada: " + MANIFEST_OF_JOBS[0]["query_string"])

2025-10-08 20:04:14,375 - INFO - Definiendo el sistema escalable de consultas geoespaciales.
2025-10-08 20:04:14,402 - INFO - Manifiesto de trabajo con 2706 consultas guardado exitosamente en: ../datos_brutos/api/manifest.json
2025-10-08 20:04:14,403 - INFO - Ejemplo de consulta final generada: restaurantes gourmet en la ciudad llamada Buenaventura, Valle del Cauca, Colombia


In [138]:
# --- Celda 6: Ejecución de la Extracción por Descubrimiento (Versión Extendida) ---
def load_processed_ids_from_csv():
    """Función auxiliar para leer los IDs del CSV si el estado no existe."""
    try:
        if os.path.exists(RAW_API_CSV_PATH):
            df_existente = pd.read_csv(RAW_API_CSV_PATH)
            logging.info(f"Archivo CSV existente encontrado. Cargando {len(df_existente['place_id'].unique())} IDs ya procesados.")
            return set(df_existente['place_id'].dropna().unique())
    except Exception as e:
        logging.error(f"No se pudo leer el archivo CSV para cargar IDs previos: {e}")
    return set()

def load_state():
    """Carga el estado desde JSON. Si no existe, intenta cargar desde el CSV."""
    if os.path.exists(STATE_FILE_PATH):
        with open(STATE_FILE_PATH, 'r') as f:
            state = json.load(f)
            # Aseguramos que los IDs estén en un set para búsquedas rápidas
            state['processed_place_ids'] = set(state.get('processed_place_ids', []))
            return state
    # Si no hay archivo de estado, creamos uno a partir del CSV existente
    return {'processed_place_ids': load_processed_ids_from_csv()}

def save_state(state):
    """Guarda el estado a JSON, convirtiendo el set a lista para la serialización."""
    state_to_save = state.copy()
    state_to_save['processed_place_ids'] = list(state.get('processed_place_ids', []))
    with open(STATE_FILE_PATH, 'w') as f:
        json.dump(state_to_save, f, indent=4)

def run_extraction_from_manifest():
    logging.info("--- INICIANDO EXTRACCIÓN BASADA EN MANIFIESTO ---")

    # 1. Cargar el manifiesto de trabajo
    try:
        with open(MANIFEST_FILE_PATH, 'r', encoding='utf-8') as f:
            jobs_to_run = json.load(f)
        logging.info(f"Manifiesto cargado. Se procesarán {len(jobs_to_run)} consultas.")
    except FileNotFoundError:
        logging.error(f"No se encontró el archivo manifest.json en {MANIFEST_FILE_PATH}.")
        return # Detiene la ejecución si no hay manifiesto

    # 2. Cargar el estado actual (IDs ya procesados)
    state = load_state()
    processed_place_ids = state.get('processed_place_ids', set())
    
    file_exists = os.path.exists(RAW_API_CSV_PATH)
    total_jobs = len(jobs_to_run)
    
    try:
        # 3. Iterar sobre los "trabajos" del manifiesto
        for i, job in enumerate(jobs_to_run):
            # Desempaquetamos toda la información del trabajo
            query_string = job["query_string"]
            pais = job["pais"]
            departamento = job["departamento_estado"]
            ciudad = job["ciudad"]
            pueblo = job["pueblo"]
            
            logging.info(f"\nProcesando trabajo ({i+1}/{total_jobs}): '{query_string}'")
            
            place_ids_found = text_search_paginated(query_string)
            new_places_found = 0
            
            for place_id in place_ids_found:
                # La lógica de verificación de place_id se mantiene intacta
                if place_id in processed_place_ids:
                    logging.debug(f"Place ID {place_id} ya procesado. Omitiendo detalles.")
                    continue

                new_places_found += 1
                logging.info(f"Nuevo lugar encontrado! Obteniendo detalles para Place ID: {place_id}")
                
                details = get_place_details(place_id)
                
                if details:
                    # Añadimos la nueva columna 'departamento_estado'
                    reviews_text = [r.get('text', '') for r in details.get('reviews', []) if r.get('text')]
                    place_data = {
                        'pais': pais,
                        'departamento_estado': departamento,
                        'ciudad': ciudad,
                        'pueblo': pueblo,
                        'place_id': details.get('place_id'),
                        'nombre_google': details.get('name'),
                        'direccion': details.get('formatted_address'),
                        'latitud': details.get('geometry', {}).get('location', {}).get('lat'),
                        'longitud': details.get('geometry', {}).get('location', {}).get('lng'),
                        'categorias_google': ', '.join(details.get('types', [])),
                        'calificacion': details.get('rating'),
                        'total_calificaciones': details.get('user_ratings_total'),
                        'comentarios': " | ".join(reviews_text),
                        'nivel_precios': details.get('price_level'),
                        'horarios': json.dumps(details.get('opening_hours', {}), ensure_ascii=False),
                        'telefono': details.get('international_phone_number'),
                        'sitio_web': details.get('website'),
                        'entrada_accesible': details.get('wheelchair_accessible_entrance'),
                        'es_reservable': details.get('reservable'),
                        'estado_negocio': details.get('business_status')
                    }
                    
                    df_to_append = pd.DataFrame([place_data])
                    df_to_append.to_csv(RAW_API_CSV_PATH, mode='a', header=not file_exists, index=False)
                    file_exists = True
                    
                    processed_place_ids.add(place_id)
                else:
                    logging.error(f"No se pudieron obtener los detalles para Place ID {place_id}. Se omitirá.")
                
                time.sleep(0.5)
            
            logging.info(f"Trabajo '{query_string}' completado. Se encontraron {new_places_found} lugares nuevos.")
            
            state['processed_place_ids'] = processed_place_ids
            save_state(state)

    except KeyboardInterrupt:
        logging.warning("Extracción interrumpida por el usuario.")
    finally:
        save_state(state)
        logging.info(f"Proceso finalizado. Estado guardado. Total de lugares únicos en la base de datos: {len(processed_place_ids)}")

# --- Ejecutar la función principal ---
run_extraction_from_manifest()

2025-10-08 19:56:59,104 - INFO - --- INICIANDO EXTRACCIÓN BASADA EN MANIFIESTO ---
2025-10-08 19:56:59,110 - INFO - Manifiesto cargado. Se procesarán 2706 consultas.
2025-10-08 19:56:59,112 - INFO - 
Procesando trabajo (1/2706): 'restaurantes gourmet en la ciudad llamada Buenaventura, Colombia'
2025-10-08 19:56:59,113 - INFO - Realizando búsqueda inicial para: 'restaurantes gourmet en la ciudad llamada Buenaventura, Colombia'
2025-10-08 19:56:59,924 - INFO - Paginación: Obteniendo siguiente página de resultados...
2025-10-08 19:57:02,369 - INFO - Búsqueda para 'restaurantes gourmet en la ciudad llamada Buenaventura, Colombia' completada. Se encontraron 25 lugares.
2025-10-08 19:57:02,370 - INFO - Trabajo 'restaurantes gourmet en la ciudad llamada Buenaventura, Colombia' completado. Se encontraron 0 lugares nuevos.
2025-10-08 19:57:02,373 - INFO - 
Procesando trabajo (2/2706): 'restaurantes de comida típica en la ciudad llamada Buenaventura, Colombia'
2025-10-08 19:57:02,374 - INFO - Re

In [ ]:
# --- Carga del Dataset ---
try:
    data_folder = '../datos_brutos/api/'
    file_name = '001_Local_cali_api.csv'
    file_path = os.path.join(data_folder, file_name)

    df_api = pd.read_csv(file_path)
    logging.info(f"Dataset '{file_name}' cargado exitosamente.")
except FileNotFoundError:
    logging.error(f"Error: El archivo no se encontró en la ruta: {file_path}")

2025-10-08 13:46:30,469 - INFO - Dataset '001_Local_cali_api.csv' cargado exitosamente.


In [ ]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   place_id              2592 non-null   object 
 1   nombre_google         2592 non-null   object 
 2   direccion             2592 non-null   object 
 3   latitud               2592 non-null   float64
 4   longitud              2592 non-null   float64
 5   categorias_google     2592 non-null   object 
 6   calificacion          2484 non-null   float64
 7   total_calificaciones  2484 non-null   float64
 8   comentarios           2387 non-null   object 
 9   nivel_precios         405 non-null    float64
 10  horarios              2592 non-null   object 
 11  telefono              2094 non-null   object 
 12  sitio_web             1117 non-null   object 
 13  entrada_accesible     1205 non-null   object 
 14  es_reservable         426 non-null    object 
 15  estado_negocio       

In [ ]:
df_api.head(365)

,place_id,nombre_google,direccion,latitud,longitud,categorias_google,calificacion,total_calificaciones,comentarios,nivel_precios,horarios,telefono,sitio_web,entrada_accesible,es_reservable,estado_negocio
0,ChIJ4ym1rp-mMI4RdMtQYqwmVGE,La Casona Valluna,"Cra. 38d #4C - 54, Nueva Granada, Cali, Valle ...",3.426279,-76.546359,"establishment, food, point_of_interest, restau...",4.4,3865.0,"Terrible experiencia, las marranitas llegaron ...",2.0,"{""open_now"": true, ""periods"": [{""close"": {""day...",+57 310 4472694,http://www.lacasonavalluna.com/,True,True,OPERATIONAL
1,ChIJpT3NWaqnMI4RJ0B8s1ESQ40,Restaurante La Abuela,"Cl. 52 #13A-24, Comuna 8, Cali, Valle del Cauc...",3.444960,-76.499559,"establishment, food, point_of_interest, restau...",4.3,343.0,Es un restaurante el cual he conocido casi la ...,2.0,"{""open_now"": true, ""periods"": [{""close"": {""day...",+57 602 4382765,https://www.paginasamarillas.com.co/empresas/r...,NaN,False,OPERATIONAL
2,ChIJJSW8MIOmMI4Rtvn2QSI02is,La Comitiva,"Cl. 4 #3432, El Sindicato, Cali, Valle del Cau...",3.434618,-76.544993,"establishment, food, point_of_interest, restau...",4.7,2281.0,"Restaurante románticamente ambientado, situado...",3.0,"{""open_now"": false, ""periods"": [{""close"": {""da...",+57 315 0713719,https://restaurantelacomitiva.com/,True,True,OPERATIONAL
3,ChIJ66yQrX2mMI4RjMrUlemZdy0,El Zaguán de San Antonio,"Cra. 12 #1-29, San Cayetano, Cali, Valle del C...",3.446042,-76.540740,"establishment, food, point_of_interest, restau...",4.2,2600.0,Mi completa admiración hacia este lugar ya que...,2.0,"{""open_now"": false, ""periods"": [{""close"": {""da...",+57 602 8938021,https://www.facebook.com/pages/El-Zaguan-De-Sa...,False,True,OPERATIONAL
4,ChIJe3iuqS2nMI4RvZ7C18ALXFo,Mi Tierra - Restaurante colombiano,"Cl. 2 #12A-05, COMUNA 3, Cali, Valle del Cauca...",3.445091,-76.540113,"establishment, food, point_of_interest, restau...",4.5,260.0,"La comida es buena, el lugar está muy adornado...",NaN,"{""open_now"": false, ""periods"": [{""close"": {""da...",+57 301 1781322,https://www.facebook.com/mitierratradicion2017/,NaN,True,OPERATIONAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,ChIJNeouYVOmMI4RnzJwmzjYjeI,Panadería Matecaña,"Cra. 12 #42-103, Comuna 8, Cali, Valle del Cau...",3.448389,-76.504160,"bakery, establishment, food, point_of_interest...",4.3,544.0,Unos productos con muy buen sabor. | Siempre m...,NaN,"{""open_now"": false, ""periods"": [{""close"": {""da...",+57 316 5133927,NaN,NaN,NaN,OPERATIONAL
361,ChIJfXc4nHCoMI4RoVYXHG1xv9o,Panadería y Pastelería Lizpan,"COMUNA 6, Cali, Valle del Cauca, Colombia",3.488983,-76.490751,"bakery, establishment, food, point_of_interest...",4.2,526.0,"Muy buena atención, todo delicioso 🤗 | Primero...",NaN,"{""open_now"": false, ""periods"": [{""close"": {""da...",NaN,NaN,NaN,NaN,OPERATIONAL
362,ChIJq7IKZDSmMI4RMp8-Tm___3U,Panadería Brisas de Salomia,"Cra 2 #46 - 23, Esmeralda, Cali, Valle del Cau...",3.467035,-76.505836,"bakery, establishment, food, point_of_interest...",4.4,187.0,Muy buen sitio excelente comida | Es una buena...,1.0,"{""open_now"": false, ""periods"": [{""close"": {""da...",+57 602 4494529,NaN,NaN,NaN,OPERATIONAL
363,ChIJ24jxmeemMI4RFOvYfozAd1E,Panaderia La Panameña,"Cra. 32 #322, San Carlos, Cali, Valle del Cauc...",3.424586,-76.511840,"bakery, establishment, food, point_of_interest...",4.3,743.0,Si vas de paso y buscas desayunar aquí en la P...,1.0,"{""open_now"": true, ""periods"": [{""open"": {""day""...",NaN,NaN,True,NaN,OPERATIONAL
